In [1]:
!pip install xgboost
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.3.1
!pip list | grep scikit-learn
!pip install imbalanced-learn

from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import time

import numpy as np
import joblib
import pandas as pd
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix

Found existing installation: scikit-learn 1.3.1
Uninstalling scikit-learn-1.3.1:
  Successfully uninstalled scikit-learn-1.3.1
  Using cached scikit_learn-1.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.3.1 which is incompatible.
scikit-learn                       1.3.1
  Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.1
    Uninstalling scikit-learn-1.3.

In [3]:
# Load the CSV file
df = pd.read_csv('output_data.csv')

# Check the first few rows to understand the structure
import ast
print(df.head())
# Convert the "captions" column from string to list of strings
df['captions'] = df['captions'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Concatenate the captions into a unified string
df['captions_concatenated'] = df['captions'].apply(lambda captions: ' '.join(captions))

!pip install emoji

import numpy as np
import string
import unicodedata
# bert word2vec doc2vec"
import emoji, re

def preprocess_text_with_emojis(text):
    if not text or isinstance(text, float) and np.isnan(text):
        return ""

    text = text.replace('\n', ' ').replace('\r', ' ')
    text = text.replace('#', ' ').replace("'", " ").replace("’", " ").replace("\\", " ").replace("“", " ")
    text = text.replace("/", " ").replace(":", " ").replace(";", " ").replace("?", " ").replace("!", " ")
    text = text.replace("(", " ").replace(")", " ").replace("[", " ").replace("]", " ")
    text = unicodedata.normalize('NFKD', text)
    text = emoji.demojize(text, delimiters=("", " "))  # Convert emojis to descriptive text
    text = re.sub(r'[^a-zA-Z\s:üğşçöıİĞÜŞÇÖ]', '', text)  # Keep descriptive emoji text
    text = text.casefold()
    text = ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])

    return text

# Preprocess 'biography' and 'captions_concatenated' columns
df['bio'] = df['biography'].apply(preprocess_text_with_emojis)
df['cap'] = df['captions_concatenated'].apply(preprocess_text_with_emojis)



          username             label  \
0    taskirancemal  Mom and Children   
1    tam_kararinda              Food   
2         spart4nn              Food   
3  sosyalyiyiciler              Food   
4  sonaydizdarahad  Mom and Children   

                                           biography    category_name  \
0                                     📍Antalya / Kaş     Entrepreneur   
1     Milliyet Pazar\nKalori Alacaksan Buna Değecek📚  Kitchen/cooking   
2  Küçük ev\nKamp ve doğa hayatı🏕\nMutfağımız doğ...    Video creator   
3  Founder @bitte.izmir\nIZMIR yemek/seyahat food...              NaN   
4  DİZDAR 🧿 Ahad 🧿 Dağhan\n@d.a.d.kids\n@dilanpol...    Personal blog   

                                            captions  
0  ['Ah Merve ah.. #tineco #temizlik', 'Çocukken ...  
1  ['Faydasız Deneyler Serisi’nde bu sefer blende...  
2  ['Kütük ocağında et fajita...\n#nature#dinner#...  
3  ['Keşif gibi keşifle geldim!cosi.gastro Çakall...  
4  ['Her şeye EYVALLAH 🤐', '🤍', '10.yıl ❤️', 

In [4]:

# Show the result
print(df[['biography', 'bio', 'captions_concatenated', 'cap']].head())

df = df.drop(columns=['biography','captions', 'captions_concatenated'])

# Save the processed DataFrame to a new CSV file
df.to_csv('train_temizledim_with_stopWords_ama.csv', index=False)
time.sleep(10)
# Show the first few rows of the modified DataFrame to confirm
print("head:\n",df.head())

import pandas as pd
import nltk
from nltk.corpus import stopwords
import csv
csv.field_size_limit(10**6)  # Set an appropriate limit

nltk.download('stopwords')


# Get the Turkish stopwords list
turkish_stopwords = stopwords.words('turkish')

# Function to remove stopwords
def remove_stopwords(text):

    if not isinstance(text, str):
        text = ""
    # Split text into words
    words = text.split()
    # Remove stopwords and return the cleaned text
    cleaned_text = ' '.join([word for word in words if word not in turkish_stopwords])
    return cleaned_text


# Remove stopwords from 'biography_preprocessed' and 'captions_concatenated_preprocessed'
df['bio_noSw'] = df['bio'].apply(remove_stopwords)
df['cap_noSw'] = df['cap'].apply(remove_stopwords)

# Show the original and stopwords-removed columns side by side for comparison
print(df[['bio', 'bio_noSw',
          'cap', 'cap_noSw']].head())

df.to_csv('train_temiz_noSw.csv', index=False)
time.sleep(10)


df = pd.read_csv('train_temiz_noSw.csv')
df['bio_noSw'] = df['bio_noSw'].fillna("").astype(str)
df['cap_noSw'] = df['cap_noSw'].fillna("").astype(str)
df.drop(columns=['bio', 'cap'], inplace=True)
df.rename(columns={'bio_noSw': 'bio', 'cap_noSw': 'cap'}, inplace=True)
print(df.head())

from sklearn.feature_extraction.text import TfidfVectorizer
# Initialize the vectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=3000,
    stop_words=None,
)


# Fit on training data (captions and biographies combined, or separately)
tfidf_vectorizer.fit(df['cap'] + df['bio'])  # Fit on combined text

# Transform training set
captions_tfidf = tfidf_vectorizer.transform(df['cap'])
biography_tfidf = tfidf_vectorizer.transform(df['bio'])

import pandas as pd

biography_tfidf_df = pd.DataFrame(biography_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
captions_tfidf_df = pd.DataFrame(captions_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Combine TF-IDF features with original DataFrame
df_with_tfidf = pd.concat([df, biography_tfidf_df.add_prefix('bio_'), captions_tfidf_df.add_prefix('cap_')], axis=1)

df_with_tfidf.drop(columns=['bio', 'cap'], inplace=True)
print(df_with_tfidf.head())

#df_with_tfidf.to_csv('5k_train_x_y_no_category.csv', index=False)


                                           biography  \
0                                     📍Antalya / Kaş   
1     Milliyet Pazar\nKalori Alacaksan Buna Değecek📚   
2  Küçük ev\nKamp ve doğa hayatı🏕\nMutfağımız doğ...   
3  Founder @bitte.izmir\nIZMIR yemek/seyahat food...   
4  DİZDAR 🧿 Ahad 🧿 Dağhan\n@d.a.d.kids\n@dilanpol...   

                                                 bio  \
0                         roundpushpin antalya   kas   
1  milliyet pazar kalori alacaksan buna degecekbo...   
2  kucuk ev kamp ve doga hayatıcamping  mutfagımı...   
3  founder bitteizmir izmir yemek seyahat food tr...   
4  dizdar nazaramulet  ahad nazaramulet  daghan d...   

                               captions_concatenated  \
0  Ah Merve ah.. #tineco #temizlik Çocukken baktı...   
1  Faydasız Deneyler Serisi’nde bu sefer blender’...   
2  Kütük ocağında et fajita...\n#nature#dinner#la...   
3  Keşif gibi keşifle geldim!cosi.gastro Çakallı ...   
4  Her şeye EYVALLAH 🤐 🤍 10.yıl ❤️ Şanlıurfa d

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                 bio  \
0                         roundpushpin antalya   kas   
1  milliyet pazar kalori alacaksan buna degecekbo...   
2  kucuk ev kamp ve doga hayatıcamping  mutfagımı...   
3  founder bitteizmir izmir yemek seyahat food tr...   
4  dizdar nazaramulet  ahad nazaramulet  daghan d...   

                                            bio_noSw  \
0                           roundpushpin antalya kas   
1  milliyet pazar kalori alacaksan buna degecekbooks   
2  kucuk ev kamp doga hayatıcamping mutfagımız do...   
3  founder bitteizmir izmir yemek seyahat food tr...   
4  dizdar nazaramulet ahad nazaramulet daghan dad...   

                                                 cap  \
0  ah merve ah  tineco  temizlik cocukken baktıgı...   
1  faydasız deneyler serisi nde bu sefer blender ...   
2  kutuk ocagında et fajita  nature dinner lake c...   
3  kesif gibi kesifle geldim cosigastro cakallı m...   
4  her seye eyvallah zippermouthface  whitehea

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

#dftf = pd.read_csv('5k_train_x_y_no_category.csv')
dftf= df_with_tfidf
dftf['label'] = dftf['label'].replace('Health and lifestyle', 'Health and Lifestyle')

print(dftf[10:15])
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd

label_encoder = LabelEncoder()
label_encoder.fit(dftf["label"])

dftf["label_encoded"] = label_encoder.transform(dftf["label"])


label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print("Mapping of strings to integers:", label_mapping)





category_encoder = LabelEncoder()
category_encoder.fit(dftf["category_name"])

dftf["category_name_encoded"] = category_encoder.transform(dftf["category_name"])


category_mapping = dict(zip(category_encoder.classes_, range(len(category_encoder.classes_))))
print("Mapping of strings to integers:", category_mapping)

dftf = dftf.drop(columns=["label", "category_name"])

# Display the resulting DataFrame
print(dftf.head())


# original_labels = label_encoder.inverse_transform(labels_encoded)
# original_category_names = category_encoder.inverse_transform(df["category_name_encoded"])


             username                 label         category_name  bio_about  \
10     ruyabuyuktetik               Fashion         Personal blog        0.0   
11         raykakumru  Health and Lifestyle         Public figure        0.0   
12         pintipanda                Gaming  Gaming video creator        0.0   
13  pinarindepresyonu  Health and Lifestyle       Digital creator        0.0   
14         pinarhotic  Health and Lifestyle       Digital creator        0.0   

    bio_ac  bio_acar  bio_acil  bio_acı  bio_acık  bio_acılan  ...  \
10     0.0       0.0       0.0      0.0       0.0         0.0  ...   
11     0.0       0.0       0.0      0.0       0.0         0.0  ...   
12     0.0       0.0       0.0      0.0       0.0         0.0  ...   
13     0.0       0.0       0.0      0.0       0.0         0.0  ...   
14     0.0       0.0       0.0      0.0       0.0         0.0  ...   

    cap_zirvesi  cap_ziyaret  cap_ziyaretleri   cap_zor  cap_zorlu  \
10          0.0          0.0

In [6]:

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


# Isolate the label column
labels = dftf['label_encoded']

# Perform stratified splitting
train_df, test_df = train_test_split(
    dftf,
    test_size=0.2,  # 20% test size
    stratify=labels,
    random_state=42
)

# Verify the split
label_distribution_train = train_df['label_encoded'].value_counts(normalize=False)
label_distribution_test = test_df['label_encoded'].value_counts(normalize=False)

print("Label distribution in train set:")
print(label_distribution_train)

print("\nLabel distribution in test set:")
print(label_distribution_test)

# Check the shapes of the resulting datasets
print(f"\nTraining set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")


Label distribution in train set:
label_encoded
3    409
5    402
8    277
1    258
2    239
9    235
0    153
6    119
7     90
4     10
Name: count, dtype: int64

Label distribution in test set:
label_encoded
3    102
5    100
8     69
1     65
2     60
9     59
0     38
6     30
7     23
4      3
Name: count, dtype: int64

Training set shape: (2192, 6003)
Test set shape: (549, 6003)


In [8]:

#train_df.columns[-11:-1]
#train_df.to_csv('5ktrain_df_1hot.csv', index=False)
#
#test_df.columns[-11:-1]
#test_df.to_csv('5ktest_df_1hot.csv', index=False)
#
#train_df = pd.read_csv('5ktrain_df_1hot.csv')
#test_df = pd.read_csv('5ktest_df_1hot.csv')

#Step 2: Data Preparation
#Drop unnecessary columns:

train_X = train_df.drop(columns=['username', 'label_encoded'])
test_X = test_df.drop(columns=['username', 'label_encoded'])

print(train_X.head())
# Extract the one-hot-encoded labels for training and testing
train_y = train_df.iloc[:,-2]  # Last 11 columns for one-hot-encoded labels
test_y = test_df.iloc[:,-2]

print(train_X.head())
print(test_X.head())
print(train_y.head())
print(test_y.head())


      bio_about  bio_ac  bio_acar  bio_acil  bio_acı  bio_acık  bio_acılan  \
2600        0.0     0.0       0.0       0.0      0.0       0.0         0.0   
335         0.0     0.0       0.0       0.0      0.0       0.0         0.0   
2157        0.0     0.0       0.0       0.0      0.0       0.0         0.0   
292         0.0     0.0       0.0       0.0      0.0       0.0         0.0   
1544        0.0     0.0       0.0       0.0      0.0       0.0         0.0   

      bio_acıldı  bio_acılıs  bio_acısından  ...  cap_ziyaret  \
2600         0.0         0.0            0.0  ...     0.000000   
335          0.0         0.0            0.0  ...     0.037989   
2157         0.0         0.0            0.0  ...     0.000000   
292          0.0         0.0            0.0  ...     0.000000   
1544         0.0         0.0            0.0  ...     0.000000   

      cap_ziyaretleri  cap_zor  cap_zorlu  cap_zorunda  cap_zu  cap_ın  \
2600              0.0      0.0        0.0          0.0     0.0    

In [9]:
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42,shuffle=False)


best_params = {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 300, 'subsample': 0.7,
               'colsample_bytree': 1.0, 'gamma': 0.1, 'reg_alpha': 0, 'reg_lambda': 0.1}

# Initialize XGBoost classifier
xgb_model = XGBClassifier(
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    gamma=best_params['gamma'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'],
    #tree_method='hist',
    #device='cuda',
    eval_metric='mlogloss',
    objective='multi:softmax', # For multi-class classification
    num_class=len(np.unique(train_y)), # Number of classes
    random_state=42,

)


# Define the parameter grid to search
''' param_grid = {
    'max_depth': [4],             # Depth of trees
    'learning_rate': [0.01,0.1],  # Step size shrinkage
    'n_estimators': [200,300],     # Number of trees
   'subsample': [0.7],  # Fraction of samples for each tree
   'colsample_bytree': [1.0],# Fraction of features for each tree 7
   'gamma': [0.1],             # Minimum loss reduction for split
   'reg_alpha': [0],           # L1 regularization
   'reg_lambda': [0.1]           # L2 regularization
} '''

# Set up GridSearchCV with cross-validation
#grid_search = GridSearchCV(
#    estimator=xgb_model,
#    param_grid=param_grid,
#    scoring='accuracy',
#    cv=3,                # Number of cross-validation folds
#    verbose=2,
#    n_jobs=1
#)



# Train the model
#grid_search.fit(train_X, train_y)
#print("Best hyperparameters found: ", grid_search.best_params_)
#
## Get the best model from GridSearchCV
#best_model = grid_search.best_estimator_
#import matplotlib.pyplot as plt
#from xgboost import plot_importance
#
##plot_importance(xgb_model)
##plt.show()
## Evaluate the model with cross-validation
#from sklearn.model_selection import cross_val_score
#cv_scores = cross_val_score(xgb_model, train_X, train_y, cv=5, scoring='accuracy')
#print(f"Cross-validation scores: {cv_scores}")
#print(f"Average cross-validation accuracy: {np.mean(cv_scores)}")
#
## Predict on test set
#y_pred = best_model.predict(X_val)
#validation_accuracy = accuracy_score(y_val, y_pred)
#print(f"Validation Accuracy: {validation_accuracy}")
#
## Evaluate the best model on the test set
#y_test_pred = best_model.predict(test_X)
#test_accuracy = accuracy_score(test_y, y_test_pred)
#print(f"Test Accuracy: {test_accuracy}")
#
#print("\nClassification Report:")
#print(classification_report(test_y, y_test_pred))
#joblib.dump(xgb_model, 'xgb_model.pkl')


xgb_model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = xgb_model.predict(X_val)
validation_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {validation_accuracy}")
#
## Evaluate the model on the test set
y_test_pred = xgb_model.predict(test_X)
test_accuracy = accuracy_score(test_y, y_test_pred)
print(f"Test Accuracy: {test_accuracy}")
#
## Print classification report to assess performance
print("\nClassification Report:")
print(classification_report(test_y, y_test_pred))



Validation Accuracy: 0.6651480637813212
Test Accuracy: 0.6484517304189436

Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.39      0.45        38
           1       0.42      0.38      0.40        65
           2       0.74      0.67      0.70        60
           3       0.81      0.86      0.84       102
           4       0.00      0.00      0.00         3
           5       0.55      0.80      0.65       100
           6       0.57      0.43      0.49        30
           7       0.82      0.39      0.53        23
           8       0.68      0.75      0.72        69
           9       0.81      0.58      0.67        59

    accuracy                           0.65       549
   macro avg       0.59      0.53      0.55       549
weighted avg       0.66      0.65      0.64       549



In [10]:
test_X

,bio_about,bio_ac,bio_acar,bio_acil,bio_acı,bio_acık,bio_acılan,bio_acıldı,bio_acılıs,bio_acısından,...,cap_ziyaret,cap_ziyaretleri,cap_zor,cap_zorlu,cap_zorunda,cap_zu,cap_ın,cap_ısık,cap_ızgara,category_name_encoded
1338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,13
2723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,81
1056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,100
2423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.050543,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,250
2312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.104765,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,192
1546,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.006421,0.0,0.0,264
1751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,152
1514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012109,0.0,0.017050,0.019796,0.000000,0.0,0.078448,0.0,0.0,260


In [12]:
import pickle
import joblib
joblib.dump(xgb_model, 'xgboost_model.joblib')

['xgboost_model.joblib']

In [22]:
# testing with p2 data
df = pd.read_csv('test_datap3.csv')
print(df.head())
df['captions'] = df['captions'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Concatenate the captions into a unified string
df['captions_concatenated'] = df['captions'].apply(lambda captions: ' '.join(captions))

# Preprocess 'biography' and 'captions_concatenated' columns
df['bio'] = df['biography'].apply(preprocess_text_with_emojis)
df['cap'] = df['captions_concatenated'].apply(preprocess_text_with_emojis)



        username                                          biography  \
0  livapastanesi  Çukurambar/Farabi/A.City/Yıldız/Ayrancı/Taurus...   
1     barisgross  %100 Yerli Besi Güvenilir Et\nTaptaze Meyve v...   
2      tusasshop  Türk Havacılık Uzay Sanayii Lisanslı Ürünler...   
3  etolyadigital  #GücünüKeşfet\nMarkaları geliştiren dijital pa...   
4     tugrulonur  Genel Yayın Yönetmeni ve Moderatör @voleapp \n...   

      category_name                                           captions  
0      Dessert Shop  ['" Atatürk, bir lider olmanın çok daha ötesin...  
1    Retail company  ['Halk Günü Fırsatlarını Kaçırma !\n🍅🥬🌶️🍋🫑🌽🥑🍑\...  
2   Product/service  ['Hiç durmadan, hiç yorulmadan, daima gösterdi...  
3  Marketing Agency  ['Devamlılık başarıyı getirir. 💪\n\n#webajans ...  
4        Journalist  ['Hull City’ye uğurlu geldim. Takım, üstün oyn...  


In [23]:

# Show the result
print(df[['biography', 'bio', 'captions_concatenated', 'cap']].head())

df = df.drop(columns=['biography','captions', 'captions_concatenated'])
print("head:\n",df.head())


csv.field_size_limit(10**6)  # Set an appropriate limit

# Remove stopwords from 'biography_preprocessed' and 'captions_concatenated_preprocessed'
df['bio_noSw'] = df['bio'].apply(remove_stopwords)
df['cap_noSw'] = df['cap'].apply(remove_stopwords)

# Show the original and stopwords-removed columns side by side for comparison
print(df[['bio', 'bio_noSw',
          'cap', 'cap_noSw']].head())


df['bio_noSw'] = df['bio_noSw'].fillna("").astype(str)
df['cap_noSw'] = df['cap_noSw'].fillna("").astype(str)
df.drop(columns=['bio', 'cap'], inplace=True)
df.rename(columns={'bio_noSw': 'bio', 'cap_noSw': 'cap'}, inplace=True)
print(df.head())


# Transform test set using the same vectorizer
captions_tfidf_test = tfidf_vectorizer.transform(df['cap'])
biography_tfidf_test = tfidf_vectorizer.transform(df['bio'])

biography_tfidf_test = pd.DataFrame(biography_tfidf_test.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
captions_tfidf_test = pd.DataFrame(captions_tfidf_test.toarray(), columns=tfidf_vectorizer.get_feature_names_out())


# Combine TF-IDF features with original DataFrame
df_with_tfidf = pd.concat([df, biography_tfidf_test.add_prefix('bio_'), captions_tfidf_test.add_prefix('cap_')], axis=1)

df_with_tfidf.drop(columns=['bio', 'cap'], inplace=True)
print(df_with_tfidf.head())

dftf= df_with_tfidf


                                           biography  \
0  Çukurambar/Farabi/A.City/Yıldız/Ayrancı/Taurus...   
1  %100 Yerli Besi Güvenilir Et\nTaptaze Meyve v...   
2  Türk Havacılık Uzay Sanayii Lisanslı Ürünler...   
3  #GücünüKeşfet\nMarkaları geliştiren dijital pa...   
4  Genel Yayın Yönetmeni ve Moderatör @voleapp \n...   

                                                 bio  \
0  cukurambar farabi acity yıldız ayrancı taurus ...   
1   yerli besi guvenilir et taptaze meyve ve sebz...   
2  turk havacılık uzay sanayii lisanslı urunleri ...   
3   gucunukesfet markaları gelistiren dijital paz...   
4  genel yayın yonetmeni ve moderator voleapp  pa...   

                               captions_concatenated  \
0  " Atatürk, bir lider olmanın çok daha ötesinde...   
1  Halk Günü Fırsatlarını Kaçırma !\n🍅🥬🌶️🍋🫑🌽🥑🍑\n\...   
2  Hiç durmadan, hiç yorulmadan, daima gösterdiği...   
3  Devamlılık başarıyı getirir. 💪\n\n#webajans #s...   
4  Hull City’ye uğurlu geldim. Takım, üstün oy

In [24]:

print("check\n",dftf['category_name'].head())

# Check for unseen categories
unseen_categories = set(dftf["category_name"]) - set(category_encoder.classes_)

# Add unseen categories to encoder with a special label (333)
if unseen_categories:
    print(f"Unseen categories: {unseen_categories}")
    all_categories = list(category_encoder.classes_)
    all_categories.append("UNSEEN_CATEGORY")  # Placeholder for unseen categories
    category_encoder.classes_ = np.array(all_categories)

# Map unseen categories to 333
dftf["category_name_encoded"] = dftf["category_name"].apply(
    lambda x: 333 if x in unseen_categories else category_encoder.transform([x])[0]
)

#333 is the NaN mapping

dftf = dftf.drop(columns=["category_name"])

# Display the resulting DataFrame
print(dftf['category_name_encoded'].head())
# original_labels = label_encoder.inverse_transform(labels_encoded)
# original_category_names = category_encoder.inverse_transform(df["category_name_encoded"])


check
 0        Dessert Shop
1      Retail company
2     Product/service
3    Marketing Agency
4          Journalist
Name: category_name, dtype: object
Unseen categories: {nan, 'Hobby Store', 'Tour Guide', 'Cafeteria', 'Travel Service', 'Professional Networking', 'Lawyer & Law Firm', 'Dorm', 'Gamer', 'Esports Team', 'Media Agency', 'Podcast', 'Newspaper', 'Cultural Center', 'Pizza place', 'Dance Studio', 'Mobile Phone Shop', 'Glass Manufacturer', 'Interior Design Studio', 'Supermarket', 'Water Park', 'Home Window Service', 'Mining Company', 'Library', 'Entertainment website', 'Fitness Trainer', 'Plastic Fabricator', 'Art Gallery', 'Mental health service', 'Financial Consultant', 'Metal Supplier', 'Book', 'Music Chart', 'Big Box Retailer', 'Glass Blower', 'Movie/television studio', 'Travel & Transportation', 'Boutique Store'}
0     99
1    268
2    250
3    192
4    178
Name: category_name_encoded, dtype: int64


In [25]:
usernames_col = dftf['username']
dftf.drop(columns=['username'], inplace=True)
print(dftf)

     bio_about  bio_ac  bio_acar  bio_acil  bio_acı  bio_acık  bio_acılan  \
0          0.0     0.0       0.0       0.0      0.0       0.0         0.0   
1          0.0     0.0       0.0       0.0      0.0       0.0         0.0   
2          0.0     0.0       0.0       0.0      0.0       0.0         0.0   
3          0.0     0.0       0.0       0.0      0.0       0.0         0.0   
4          0.0     0.0       0.0       0.0      0.0       0.0         0.0   
..         ...     ...       ...       ...      ...       ...         ...   
995        0.0     0.0       0.0       0.0      0.0       0.0         0.0   
996        0.0     0.0       0.0       0.0      0.0       0.0         0.0   
997        0.0     0.0       0.0       0.0      0.0       0.0         0.0   
998        0.0     0.0       0.0       0.0      0.0       0.0         0.0   
999        0.0     0.0       0.0       0.0      0.0       0.0         0.0   

     bio_acıldı  bio_acılıs  bio_acısından  ...  cap_ziyaret  cap_ziyaretle

In [26]:
y_test_pred = xgb_model.predict(dftf)

In [27]:
predicted_labels = label_encoder.inverse_transform(y_test_pred)
print(predicted_labels)

['Food' 'Food' 'Travel' 'Tech' 'Entertainment' 'Health and Lifestyle'
 'Mom and Children' 'Fashion' 'Travel' 'Food' 'Travel'
 'Health and Lifestyle' 'Art' 'Fashion' 'Food' 'Health and Lifestyle'
 'Entertainment' 'Health and Lifestyle' 'Mom and Children' 'Fashion'
 'Food' 'Art' 'Entertainment' 'Fashion' 'Entertainment' 'Art' 'Travel'
 'Food' 'Entertainment' 'Art' 'Art' 'Food' 'Health and Lifestyle' 'Art'
 'Fashion' 'Entertainment' 'Travel' 'Food' 'Health and Lifestyle'
 'Health and Lifestyle' 'Mom and Children' 'Tech' 'Entertainment'
 'Entertainment' 'Tech' 'Health and Lifestyle' 'Health and Lifestyle'
 'Entertainment' 'Food' 'Fashion' 'Mom and Children'
 'Health and Lifestyle' 'Fashion' 'Mom and Children' 'Travel' 'Fashion'
 'Health and Lifestyle' 'Health and Lifestyle' 'Health and Lifestyle'
 'Fashion' 'Health and Lifestyle' 'Tech' 'Food' 'Health and Lifestyle'
 'Health and Lifestyle' 'Fashion' 'Entertainment' 'Fashion' 'Tech' 'Tech'
 'Food' 'Fashion' 'Health and Lifestyle' 'Fashion' 

In [19]:
"""
Mapping of strings to integers: {
  'Art': 0,
  'Entertainment': 1,
  'Fashion': 2,
  'Food': 3,
  'Gaming': 4,
  'Health and Lifestyle': 5,
  'Mom and Children': 6,
  'Sports': 7,
  'Tech': 8,
  'Travel': 9}
"""
print(y_test_pred)

[5 5 8 8 5 9 2 3 3 9 3 8 5 6 8 8 5 5 5 9 1 8 3 5 5 5 5 7 1 9 1 0 7 8 6 3 3
 3 1 1 0 5 8 5 1 3 3 2 2 2 0 1 2 5 8 9 5 1 5 5 9 3 2 2 8 5 7 3 7 7 5 0 1 6
 3 3 9 1 5 3 8 1 5 5 5 8 5 1 3 5 8 5 5 1 5 5 5 1 1 9 1 9 3 2 5 5 5 1 2 1 8
 5 5 3 3 5 5 2 1 1 9 5 8 5 0 3 5 5 5 3 3 5 9 3 1 2 5 9 9 1 6 0 5 5 0 8 3 2
 3 2 3 5 5 5 0 5 2 3 2 0 9 5 5 8 5 7 9 8 2 5 1 6 3 1 1 2 0 0 3 1 8 1 0 9 6
 1 2 8 5 5 9 0 5 5 0 8 7 2 9 5 5 7 0 2 3 5 9 2 6 0 8 5 5 5 8 2 3 2 5 1 2 2
 7 3 1 5 1 8 3 3 9 1 0 1 0 0 9 2 5 5 2 5 3 8 3 5 9 5 0 3 8 5 4 5 3 8 9 5 3
 5 5 6 5 2 3 1 3 5 5 3 8 5 8 5 3 6 1 3 1 1 3 5 5 8 8 1 8 2 5 1 0 8 1 2 6 5
 5 8 6 3 1 2 9 8 5 1 3 5 2 9 3 5 3 1 3 8 1 5 9 8 5 0 5 0 3 5 9 3 3 2 5 2 1
 2 8 5 5 5 5 8 5 8 5 3 8 3 9 5 0 3 6 5 8 9 2 2 3 1 0 5 5 0 3 5 3 3 5 6 5 5
 5 3 5 1 6 5 1 5 5 9 3 2 5 3 3 6 3 2 8 9 8 5 5 0 0 8 2 9 1 3 8 5 5 0 3 3 0
 3 1 5 0 8 1 1 9 3 5 3 3 2 5 0 5 1 5 7 5 8 1 9 2 3 5 5 3 1 9 5 3 5 3 1 9 3
 6 1 2 5 3 8 3 5 5 5 5 5 5 5 5 5 5 1 2 9 2 5 9 2 9 5 3 8 8 9 9 5 8 5 5 2 2
 5 7 3 5 5 3 5 5 5 3 0 0 

In [ ]:

"""
Progressive CVgrid runs:

Best hyperparameters found:  {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}
Best hyperparameters found:  {'learning_rate': 0.2, 'max_depth': 2, 'n_estimators': 300}
Best hyperparameters found:  {'learning_rate': 0.2, 'max_depth': 1, 'n_estimators': 300}
Best hyperparameters found:  {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 1, 'n_estimators': 300, 'subsample': 0.7}
Best hyperparameters found:  {'colsample_bytree': 1.0, 'gamma': 0.1, 'learning_rate': 0.2, 'max_depth': 1, 'n_estimators': 300, 'subsample': 0.7}
Best hyperparameters found:  {'colsample_bytree': 1.0, 'gamma': 0.1, 'learning_rate': 0.2, 'max_depth': 1, 'n_estimators': 300, 'reg_alpha': 0, 'reg_lambda': 0.1, 'subsample': 0.7}
Best hyperparameters found:  {'colsample_bytree': 1.0, 'gamma': 0.1, 'learning_rate': 0.2, 'max_depth': 1, 'n_estimators': 300, 'reg_alpha': 0, 'reg_lambda': 0.1, 'subsample': 0.7}
Best hyperparameters found:  {'colsample_bytree': 1.0, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 300, 'reg_alpha': 0, 'reg_lambda': 0.1, 'subsample': 0.7}

3k:
Best hyperparameters found:  {'colsample_bytree': 1.0, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 300, 'reg_alpha': 0, 'reg_lambda': 0.1, 'subsample': 0.7}

"""